# Initialization

In [1]:
import json
import uuid
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from kafka import KafkaProducer
from faker import Faker
from time import sleep

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("Dibimbing Spark-Kafka") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2')
    .config("spark.sql.shuffle.partitions", 4)
    .master("local[*]") 
    .getOrCreate()
)

spark

In [3]:
# streaming = (
#     spark
#     .readStream
#     .schema(dataSchema)
#     .option('maxFilesPerTrigger', 1)
#     .json('/resources/data/activity-data/')
# )

In [4]:
# # set partitions
# spark.conf.set('spark.sql.shuffle.partitions', 5)

In [5]:
# activityCounts = streaming.select('index').distinct()
# activityQuery = (
#     activityCounts.writeStream
#     .queryName('activity_counts_3')
#     .format('memory')
#     .outputMode('append')
#     .start()
# )

# # activityQuery.awaitTermination()

In [6]:
# # activityQuery.awaitTermination()
# activityQuery.stop()

In [7]:
# from time import sleep
# for x in range(5):
#     spark.sql("SELECT COUNT(*) FROM activity_counts_3").show()
#     sleep(1)

# Spark - Kafka Streaming

In [8]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [9]:
kafka_host = os.getenv('KAFKA_HOST')
kafka_topic = os.getenv('KAFKA_TOPIC_NAME')
kafka_topic_partition = os.getenv('KAFKA_TOPIC_NAME')+"-1"

## Batch Simulation

In [10]:
kafka_df = (
    spark
    .read
    .format("kafka")
    .option("kafka.bootstrap.servers", f'{kafka_host}:9092')
    .option("subscribe", kafka_topic)
    .option("startingOffsets", "earliest")
    .load()
)

In [11]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [12]:
kafka_df.show()

+----+--------------------+--------------------+---------+------+--------------------+-------------+
| key|               value|               topic|partition|offset|           timestamp|timestampType|
+----+--------------------+--------------------+---------+------+--------------------+-------------+
|null|[7B 22 74 72 61 6...|apotek-dibimbing-...|        2|   252|2025-01-25 12:01:...|            0|
|null|[7B 22 74 72 61 6...|apotek-dibimbing-...|        2|   253|2025-01-25 12:01:...|            0|
|null|[7B 22 74 72 61 6...|apotek-dibimbing-...|        2|   254|2025-01-25 12:01:...|            0|
|null|[7B 22 74 72 61 6...|apotek-dibimbing-...|        2|   255|2025-01-25 12:02:...|            0|
|null|[7B 22 74 72 61 6...|apotek-dibimbing-...|        2|   256|2025-01-25 12:02:...|            0|
|null|[7B 22 74 72 61 6...|apotek-dibimbing-...|        2|   257|2025-01-25 12:04:...|            0|
|null|[7B 22 74 72 61 6...|apotek-dibimbing-...|        2|   258|2025-01-25 12:04:...|     

In [13]:
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [14]:
kafka_json_df.show(5)

+----+--------------------+--------------------+---------+------+--------------------+-------------+
| key|               value|               topic|partition|offset|           timestamp|timestampType|
+----+--------------------+--------------------+---------+------+--------------------+-------------+
|null|{"transaction_id"...|apotek-dibimbing-...|        2|   252|2025-01-25 12:01:...|            0|
|null|{"transaction_id"...|apotek-dibimbing-...|        2|   253|2025-01-25 12:01:...|            0|
|null|{"transaction_id"...|apotek-dibimbing-...|        2|   254|2025-01-25 12:01:...|            0|
|null|{"transaction_id"...|apotek-dibimbing-...|        2|   255|2025-01-25 12:02:...|            0|
|null|{"transaction_id"...|apotek-dibimbing-...|        2|   256|2025-01-25 12:02:...|            0|
+----+--------------------+--------------------+---------+------+--------------------+-------------+
only showing top 5 rows



In [15]:
(
    kafka_json_df
    .select('value')
    .limit(5)
    .collect()
)

[Row(value='{"transaction_id": "98092f1b-6da2-4efc-abd2-61ea90f7e5bd", "buyer_name": "James Ward", "medication_name": "Paracetamol", "quantity": 5, "unit_price": 4106, "total_price": 20530, "payment_method": "Cash", "transaction_date": "2025-01-22", "transaction_time": "21:53:12", "ts": 1737805179}'),
 Row(value='{"transaction_id": "e3679d58-6d7b-4a0c-8e95-caae6ec41775", "buyer_name": "Brian Parker", "medication_name": "Cough Syrup", "quantity": 2, "unit_price": 14700, "total_price": 29400, "payment_method": "Cash", "transaction_date": "2025-01-04", "transaction_time": "04:37:14", "ts": 1737806503}'),
 Row(value='{"transaction_id": "c49ae644-3569-41ca-bcbb-24f28d1d2461", "buyer_name": "David Dixon", "medication_name": "Antibiotik", "quantity": 2, "unit_price": 6861, "total_price": 13722, "payment_method": "QRIS", "transaction_date": "2025-01-21", "transaction_time": "18:02:53", "ts": 1737803917}'),
 Row(value='{"transaction_id": "a280a4e4-cfd9-467c-ad09-16e5f3c612d9", "buyer_name": "Be

In [16]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType


schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("buyer_name", StringType(), True),
    StructField("medication_name", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("unit_price", IntegerType(), True),
    StructField("total_price", IntegerType(), True),
    StructField("payment_method", StringType(), True),
    StructField("transaction_date", StringType(), True),
    StructField("transaction_time", StringType(), True),
    StructField("ts", StringType(), True),
])


In [17]:
from pyspark.sql.functions import from_json, col

(
    kafka_json_df
    .select(
        from_json(col("value"), schema)
        .alias("data")
    )
    .select("data.*")
    .show()
)

+--------------------+------------------+---------------+--------+----------+-----------+--------------+----------------+----------------+----------+
|      transaction_id|        buyer_name|medication_name|quantity|unit_price|total_price|payment_method|transaction_date|transaction_time|        ts|
+--------------------+------------------+---------------+--------+----------+-----------+--------------+----------------+----------------+----------+
|98092f1b-6da2-4ef...|        James Ward|    Paracetamol|       5|      4106|      20530|          Cash|      2025-01-22|        21:53:12|1737805179|
|e3679d58-6d7b-4a0...|      Brian Parker|    Cough Syrup|       2|     14700|      29400|          Cash|      2025-01-04|        04:37:14|1737806503|
|c49ae644-3569-41c...|       David Dixon|     Antibiotik|       2|      6861|      13722|          QRIS|      2025-01-21|        18:02:53|1737803917|
|a280a4e4-cfd9-467...|        Beth Gomez|    Amoxicillin|       2|     16352|      32704|    Debit C

## Stream Simulation

In [18]:
kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", f'{kafka_host}:9092')
    .option("subscribe", kafka_topic)
    .option("startingOffsets", "earliest")
    .load()
)

In [19]:
from pyspark.sql.functions import from_json, col

parsed_df = (
    kafka_df
    .withColumn("value", expr("cast(value as string)"))
    .select(
        from_json(col("value"), schema)
        .alias("data")
    )
    .select("data.*")
)

In [ ]:
from pyspark.sql.functions import col, from_unixtime, sum,to_timestamp


# parsed_df = parsed_df.withColumn("ts", from_unixtime(col("ts")))
# parsed_df = parsed_df.withColumn("ts", to_timestamp(col("transaction_time"), "yyyy-MM-dd HH:mm:ss"))

parsed_df.printSchema()

# Konversi kolom transaction_time ke timestamp
parsed_df = parsed_df.withColumn("transaction_time", to_timestamp(col("transaction_time"), "yyyy-MM-dd HH:mm:ss"))

# Periksa schema setelah konversi
parsed_df.printSchema()

# Menambahkan watermark berdasarkan kolom 'transaction_time'
parsed_df_with_watermark = parsed_df.withWatermark("transaction_time", "10 minutes") # untuk toleransi keterlambatan data


# Agregasi data menggunakan groupBy dan sum
aggregated_df = parsed_df_with_watermark.groupBy("payment_method").agg(sum("total_price").alias("total_sales"))

(
aggregated_df.writeStream
    .format("console") 
    .outputMode("complete")
    .trigger(processingTime='5 minutes') # men-trigger event setiap waktu tertentu
    .option('checkpointlocation', '/resources/logs')
    .start()
    .awaitTermination()
)



root
 |-- transaction_id: string (nullable = true)
 |-- buyer_name: string (nullable = true)
 |-- medication_name: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit_price: integer (nullable = true)
 |-- total_price: integer (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- transaction_time: string (nullable = true)
 |-- ts: timestamp (nullable = true)

root
 |-- transaction_id: string (nullable = true)
 |-- buyer_name: string (nullable = true)
 |-- medication_name: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit_price: integer (nullable = true)
 |-- total_price: integer (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- transaction_time: timestamp (nullable = true)
 |-- ts: timestamp (nullable = true)



In [ ]:
# (
#     parsed_df
#     .writeStream
#     .format("console")
#     .outputMode("append")  # Gunakan 'append' jika tidak ada agregasi
#     # .trigger(processingTime='5 seconds')
#     # .trigger(continuous='1 second')
#     # .trigger(once=true)
#     .option("checkpointLocation", "checkpoint_dir")
#     # .option("failOnDataLoss", "false")  # Tambahkan ini
#     .start()
#     .awaitTermination()
# )